In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv("features.csv", index_col = "match_id")
df1 = pd.read_csv("features_test.zip", index_col = "match_id")
df

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,4,3,0,-24.0,2032,0,1792,1975,48,63
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,3,2,0,-17.0,1734,1,2038,6,63,3
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,1,3,1,-15.0,2906,0,1796,1846,51,63


In [3]:
df = df.drop("tower_status_radiant", axis = 1)
df = df.drop("tower_status_dire", axis = 1)
df = df.drop("barracks_status_radiant", axis = 1)
df = df.drop("barracks_status_dire", axis = 1)

### 1.Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

In [4]:
colna = []
for i in df.columns:
    if df[i].isna().sum() != 0:
        colna.append(i)
        print(i)
#

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


In [5]:
colna = []
for i in df1.columns:
    if df1[i].isna().sum() != 0:
        colna.append(i)
        print(i)
#

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


### 2.Как называется столбец, содержащий целевую переменную?

 radiant_win

In [6]:
for i in colna:
    df[i] = df[i].fillna(df[i].mean())

In [7]:
for i in colna:
    df1[i] = df1[i].fillna(df1[i].mean())

### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти выше по тексту. Какое качество при этом получилось?

In [8]:
y = df["radiant_win"]
X = df[df1.columns]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [10]:
start = time.time()
grad = GradientBoostingClassifier(n_estimators = 30)
kf = KFold(n_splits = 5, shuffle = True )
cross = cross_val_score(grad, X_train, y_train, cv = kf, scoring = "roc_auc")
print(time.time() - start)


139.24571204185486


In [11]:
cross.mean()

0.6886026576481709

### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?

In [12]:
grad = GradientBoostingClassifier(n_estimators = 40)
cross = cross_val_score(grad, X_train, y_train, cv = kf, scoring = "roc_auc")

In [13]:
cross.mean()

0.6926402850437843

### 1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем можно объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [14]:
standard = StandardScaler()

In [15]:
X_train = standard.fit_transform(X_train)
X_test = standard.transform(X_test)

In [16]:
start = time.time()
logreg = LogisticRegression()
cross = cross_val_score(logreg, X_train, y_train, cv = kf, scoring = "roc_auc")
print(time.time() - start)

3.587059736251831


In [17]:
cross.mean()

0.7164707398680124

In [18]:
df2 = df.copy()
df2

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,103.000000,-84.0,221.000000,3,4,2,2,-52.0,2874,1
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,149.000000,-84.0,195.000000,5,4,3,1,-5.0,2463,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,45.000000,-77.0,221.000000,3,4,3,1,13.0,2130,0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,124.000000,-80.0,184.000000,0,4,2,0,27.0,1459,0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,182.000000,-80.0,225.000000,6,3,3,0,-16.0,2449,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,180.000000,-76.0,180.000000,3,4,3,0,-24.0,2032,0
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,127.215028,-82.0,214.870536,4,3,2,0,-17.0,1734,1
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,249.000000,-70.0,214.870536,1,1,3,1,-15.0,2906,0


### 2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем можно объяснить это изменение?

In [19]:
df2 = df2.drop("lobby_type", axis = 1)
df2 = df2.drop("r1_hero", axis = 1)
df2 = df2.drop("r2_hero", axis = 1)
df2 = df2.drop("r3_hero", axis = 1)
df2 = df2.drop("r4_hero", axis = 1)
df2 = df2.drop("r5_hero", axis = 1)
df2 = df2.drop("d1_hero", axis = 1)
df2 = df2.drop("d2_hero", axis = 1)
df2 = df2.drop("d3_hero", axis = 1)
df2 = df2.drop("d4_hero", axis = 1)
df2 = df2.drop("d5_hero", axis = 1)
df2 = df2.drop("radiant_win", axis = 1)

In [20]:
X = df[df2.columns]
y = df["radiant_win"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [22]:
X_train = standard.fit_transform(X_train)
X_test = standard.transform(X_test)

In [23]:
start = time.time()
logreg = LogisticRegression()
cross = cross_val_score(logreg, X_train, y_train, cv = kf, scoring = "roc_auc")
print(time.time() - start)

3.4075660705566406


In [24]:
cross.mean()

0.7192436368594636

### 3. Сколько различных идентификаторов героев существует в данной игре?

In [25]:
df["r1_hero"].value_counts()

11     5045
7      3899
72     3551
39     3341
112    3119
       ... 
66      108
80       88
103      75
58       69
109      20
Name: r1_hero, Length: 108, dtype: int64

108 

### 4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем можно это объяснить?

In [26]:
y = df["radiant_win"]
X = df[df1.columns]

In [27]:
X = X.drop("lobby_type", axis = 1)
X = X.drop("r1_hero", axis = 1)
X = X.drop("r2_hero", axis = 1)
X = X.drop("r3_hero", axis = 1)
X = X.drop("r4_hero", axis = 1)
X = X.drop("r5_hero", axis = 1)
X = X.drop("d1_hero", axis = 1)
X = X.drop("d2_hero", axis = 1)
X = X.drop("d3_hero", axis = 1)
X = X.drop("d4_hero", axis = 1)
X = X.drop("d5_hero", axis = 1)

In [28]:
X_sc = standard.fit_transform(X)
X_sc = pd.DataFrame(X_sc)

In [29]:
N = 108
X_pick = np.zeros((df.shape[0], N+4))

for i, match_id in enumerate(df.index):
    for p in range(5):
        X_pick[i, df.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [30]:
X_pick = pd.DataFrame(X_pick)

In [31]:
for i in X_pick.columns:
    if X_pick[i].sum() == 0:
        print(i)
        X_pick = X_pick.drop(i, axis = 1)

23
106
107
110


In [32]:
X_pick = X_pick.reset_index()
X_sc = X_sc.reset_index()

In [33]:
X = pd.concat([X_sc, X_pick], axis = 1).reset_index()
X = X.drop("level_0", axis = 1)
X = X.drop("index", axis = 1)

In [392]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [34]:
start = time.time()
logreg = LogisticRegression(max_iter = 300)
cross = cross_val_score(logreg, X, y, cv = kf, scoring = "roc_auc")
time.time() - start

20.916301250457764

In [393]:
cv = CountVectorizer()

In [247]:
X_train= cv.fit_transform(X_train)
X_test= cv.transform(X_test)

In [35]:
cross.mean()

0.7519188711245685

### 5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [76]:
df1

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.000000,-86.0,272.000000,3,4,2,0,118.00000
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.000000,-54.0,214.726073,3,2,2,1,16.00000
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.000000,-87.0,186.000000,1,3,3,0,-34.00000
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.000000,-89.0,210.000000,3,4,2,1,-26.00000
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.000000,-86.0,180.000000,1,3,2,1,-33.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114369,1450212780,7,11,5,2054,1941,27,0,1,8,...,1,8.0,253.000000,-87.0,214.726073,4,3,2,1,-33.00000
114377,1450222875,1,3,3,748,605,1,0,0,12,...,1,-1.0,133.000000,-85.0,184.000000,2,3,4,1,-18.00000
114378,1450223593,1,85,2,575,499,0,0,0,8,...,0,20.0,133.000000,-88.0,239.000000,4,4,4,0,-36.00000


In [77]:
X_test = df1

In [78]:
X_test = X_test.drop("r1_hero", axis = 1)
X_test = X_test.drop("r2_hero", axis = 1)
X_test = X_test.drop("r3_hero", axis = 1)
X_test = X_test.drop("r4_hero", axis = 1)
X_test = X_test.drop("r5_hero", axis = 1)

X_test = X_test.drop("d1_hero", axis = 1)
X_test = X_test.drop("d2_hero", axis = 1)
X_test = X_test.drop("d3_hero", axis = 1)
X_test = X_test.drop("d4_hero", axis = 1)
X_test = X_test.drop("d5_hero", axis = 1)

In [79]:
X_test = standard.fit_transform(X_test)
X_test = pd.DataFrame(X_test)

In [80]:
N = 108
X_pick = np.zeros((df1.shape[0], N+4))

for i, match_id in enumerate(df1.index):
    for p in range(5):
        X_pick[i, df1.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df1.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [81]:
X_pick = pd.DataFrame(X_pick)

In [82]:
for i in X_pick.columns:
    if X_pick[i].sum() == 0:
        print(i)
        X_pick = X_pick.drop(i, axis = 1)

23
106
107
108
110


In [83]:
X_test = pd.concat([X_test, X_pick], axis = 1)
X_test

,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,109,111
0,-2.514875,-0.925258,0.474746,-0.237743,-0.137094,-0.370005,-0.532732,1.003884,0.305165,-0.368985,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-2.513895,-0.572547,-1.365630,-1.207913,-1.255660,-1.145090,-0.532732,-0.593200,0.305165,0.567766,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2.512377,-0.572547,-1.365630,-0.862057,-0.742715,-1.145090,-0.532732,-0.593200,1.898546,-1.305736,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-2.508381,-0.572547,-0.445442,-0.938323,-0.537967,-1.145090,0.968865,1.003884,1.101855,-1.305736,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
4,-2.507087,-0.572547,0.474746,0.038942,-1.059534,-0.812911,-0.532732,-0.593200,0.305165,1.504517,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17172,1.078106,1.543719,1.394934,1.448970,1.699166,1.733795,-0.532732,1.003884,-0.093180,0.567766,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17173,1.079926,-0.572547,-0.445442,-0.867378,-1.180227,-1.145090,-0.532732,-0.593200,1.500200,-0.368985,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
17174,1.080056,-0.572547,-1.365630,-1.174215,-1.408682,-1.255816,-0.532732,-0.593200,-0.093180,-0.368985,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.0
17175,1.083875,-0.925258,0.474746,1.076510,0.050412,-0.370005,0.968865,2.600967,-0.093180,-0.368985,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0


In [84]:
logreg = LogisticRegression(max_iter = 300)
logreg.fit(X, y)
logreg.predict_proba(X_test)

array([[0.41456882, 0.58543118],
       [0.57829561, 0.42170439],
       [0.58952043, 0.41047957],
       ...,
       [0.63995682, 0.36004318],
       [0.66126855, 0.33873145],
       [0.56492044, 0.43507956]])

In [87]:
pred = logreg.predict_proba(X_test)

In [88]:
pred.max()

0.9853208676883618

In [90]:
pred.min()

0.014679132311638171